# DSC 198 Soccer: Web Scraper

## Import Packages

In [1]:
#tabular formatting + functions and calculations
import pandas as pd
import numpy as np

#web scraping
import requests
from bs4 import BeautifulSoup

In [38]:
home_URL = 'https://ucsdtritons.com/'
schedule_URL = 'https://ucsdtritons.com/sports/womens-soccer/schedule/2019'
box_score_ID = 'sidearm-schedule-game-links-boxscore'

In [4]:
page = requests.get(schedule_URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [36]:
box_score_suffixes = []
game_list = []

for game in soup.find_all("li", class_ = box_score_ID):
    
    try:
        suffix = game.find('a').get('href')
        game_opp = game.find('a').get('aria-label')
        
        if suffix not in box_score_suffixes:
            box_score_suffixes.append(suffix)
        
        if game_opp not in game_list:
            game_list.append(game_opp)
            
    except:
        continue

In [66]:
times = []
opps = []

for game in game_list:
    time = game.split(' on ')[1]
    times.append(time)
    
    try:
        opponent = game.split(' on ')[0].split(' vs ')[1]
        opps.append(opponent)
        
    except:
        opponent = game.split(' on ')[0].split(' at ')[1]
        opps.append(opponent)

In [68]:
df = pd.DataFrame()

df['Opponent'] = opps
df['DateTime'] = times
df['Link'] = box_score_suffixes

df.head()

,Opponent,DateTime,Link
0,Concordia (Irvine),"September 5, 2019 at 7:00 PM",/sports/womens-soccer/stats/2019/concordia-irv...
1,Biola,"September 7, 2019 at 4:30 PM",/sports/womens-soccer/stats/2019/biola/boxscor...
2,Azusa Pacific,"September 14, 2019 at 7:30 PM",/sports/womens-soccer/stats/2019/azusa-pacific...
3,Simon Fraser,"September 16, 2019 at 11:30 AM",/sports/womens-soccer/stats/2019/simon-fraser/...
4,Cal State San Marcos,"September 19, 2019 at 12:30 PM",/sports/womens-soccer/stats/2019/cal-state-san...


In [70]:
a1 = df.iloc[0]['Link']
a2 = home_URL + a1
a2

'https://ucsdtritons.com//sports/womens-soccer/stats/2019/concordia-irvine-/boxscore/2931'

In [71]:
page = requests.get(a2)
soup = BeautifulSoup(page.content, 'html.parser')

In [181]:
a = soup.find(id = "period-7")

if a:
    print('not empty')
    
b = 'period-2'

In [174]:
#soup.find(id = "period-1").find_all("td")

timer = True
timestamps = []
events = []

for table_entry in soup.find(id = b).find_all("td"):
    
    if table_entry.get('aria-hidden'):
        continue
        
    table_text = table_entry.text.strip()
    
    if len(table_text) > 2:
        
        if timer:
            timestamps.append(table_text)
            timer = False
            
        else:
            events.append(table_text)
            timer = True

In [175]:
def get_minutes(timestamp):
    return timestamp.split(':')[0]

def get_seconds(timestamp):
    return timestamp.split(':')[1]

In [176]:
event_df = pd.DataFrame()
event_df['Timestamps'] = timestamps
event_df['Event'] = events

event_df['Minute'] = event_df['Timestamps'].apply(get_minutes)
event_df['Second'] = event_df['Timestamps'].apply(get_seconds)

event_df.head()

,Timestamps,Event,Minute,Second
0,45:00,Start of 2nd period [45:00].,45,00
1,45:00,"FOR CUI: , #00 Whitcomb, Kaelyn, #7 Khoury, Ol...",45,00
2,45:00,"FOR UCSD: , #15 Kimball, Kelsey, #1 Ramos, Ang...",45,00
3,45:00,"UCSD substitution: Ray, Marissa for Celeste, M...",45,00
4,45:00,"UCSD substitution: Bruno, Sophia for Kolarik, ...",45,00
